In [170]:
%%writefile mapper.py
#!/opt/bitnami/python/bin/python
# -*-coding:utf-8 -*
import sys

for line in sys.stdin: # reads from stdin
    line = line.strip()
    words = line.split()
    for word1 in words:
        for word2 in words:
            if(word1 != word2):
                print(word1+"\t"+word2)

Overwriting mapper.py


In [193]:
%%writefile reducer.py
#!/opt/bitnami/python/bin/python
# -*-coding:utf-8 -*

import sys

word2_count = 0
word1_count = 0
key = None
value = None
values = dict()
for line in sys.stdin:
    line = line.strip()
#     print(line)
    word1, word2 = line.split()
    count = 1

    # new key value pair
    if key is None:
        key = word1
        value = word2
    # same as previous pair
    if word1 == key and word2 == value:
        word2_count += count
        word1_count += count
    # different value, save the old value counts
    if word1 == key and word2 != value:
        word1_count += count
        values[value] = word2_count
        value = word2
        word2_count = count
    # new key, print the old key values
    elif word1 != key:
        values[value] = word2_count
        for item in values:
            print(key+"\t"+item+"\t"+str(values[item]/word1_count))
        values.clear()
        key = word1
        value = word2
        word2_count = count
        word1_count = count

if key is not None:
    values[value] = word2_count
    for item in values:
            print(key+"\t"+item+"\t"+str(values[item]/word1_count))

Overwriting reducer.py


In [152]:
!cat data/co.txt 

cheese	milk	1
cheese	bread	1
cheese	eggs	1
cheese	coffee	1
cheese	milk	1
milk	cheese	1
milk	eggs	1
milk	coffee	1
milk	coke	1
milk	candy	1
milk	bread	1
milk	cheese	1

In [189]:
!cat data/co.txt | python reducer.py

cheese	milk	0.2
cheese	bread	0.2
cheese	eggs	0.2
cheese	coffee	0.2
milk	cheese	0.14285714285714285
milk	eggs	0.14285714285714285
milk	coffee	0.14285714285714285
milk	coke	0.14285714285714285
milk	candy	0.14285714285714285
milk	bread	0.14285714285714285


In [200]:
!hadoop fs -rm -R /inputs
!hadoop fs -rm -R /outputs
!hadoop fs -mkdir -p /inputs
!hadoop fs -mkdir -p /outputs
!hadoop fs -put data /inputs

Deleted /inputs
rm: `/outputs': No such file or directory
2020-10-29 05:21:35,643 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2020-10-29 05:21:37,080 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2020-10-29 05:21:37,869 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false


In [5]:
!hadoop fs -ls /inputs/data

Found 3 items
-rw-r--r--   3 root supergroup        164 2020-10-29 05:21 /inputs/data/co.txt
-rw-r--r--   3 root supergroup         59 2020-10-29 05:21 /inputs/data/cosmall.txt
-rw-r--r--   3 root supergroup    4167490 2020-10-29 05:21 /inputs/data/retail.dat


# Small Data

In [1]:
!hadoop fs -rm -R /outputs
!hadoop jar /opt/hadoop-3.2.1/share/hadoop/tools/lib/hadoop-streaming-3.2.1.jar \
    -D stream.num.map.output.key.fields=2 \
    -D mapreduce.partition.keypartitioner.options=-k1,1 \
    -D mapreduce.job.reduces=7 \
    -file $PWD/mapper.py\
    -file $PWD/reducer.py\
    -mapper mapper.py \
    -reducer reducer.py \
    -input /inputs/data/cosmall.txt \
    -output /outputs/ \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner

Deleted /outputs
2020-10-29 21:14:47,366 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/training/mapper.py, /training/reducer.py, /tmp/hadoop-unjar3417544907767511383/] [] /tmp/streamjob8113371379460189818.jar tmpDir=null
2020-10-29 21:14:49,621 INFO client.RMProxy: Connecting to ResourceManager at resourcemanager/172.18.0.2:8032
2020-10-29 21:14:50,193 INFO client.AHSProxy: Connecting to Application History server at historyserver/172.18.0.3:10200
2020-10-29 21:14:50,282 INFO client.RMProxy: Connecting to ResourceManager at resourcemanager/172.18.0.2:8032
2020-10-29 21:14:50,283 INFO client.AHSProxy: Connecting to Application History server at historyserver/172.18.0.3:10200
2020-10-29 21:14:50,874 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1604005958251_0001
2020-10-29 21:14:51,642 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localH

In [4]:
!hadoop fs -ls /outputs

Found 9 items
-rw-r--r--   3 root supergroup          0 2020-10-29 21:27 /outputs/_SUCCESS
drwxr-xr-x   - root supergroup          0 2020-10-29 21:27 /outputs/_temporary
-rw-r--r--   3 root supergroup    9404342 2020-10-29 21:32 /outputs/part-00000
-rw-r--r--   3 root supergroup         95 2020-10-29 21:26 /outputs/part-00001
-rw-r--r--   3 root supergroup         30 2020-10-29 21:26 /outputs/part-00002
-rw-r--r--   3 root supergroup         32 2020-10-29 21:26 /outputs/part-00003
-rw-r--r--   3 root supergroup         91 2020-10-29 21:26 /outputs/part-00004
-rw-r--r--   3 root supergroup          0 2020-10-29 21:26 /outputs/part-00005
-rw-r--r--   3 root supergroup         29 2020-10-29 21:27 /outputs/part-00006


In [3]:
!hadoop fs -cat /outputs/part-00000
!hadoop fs -cat /outputs/part-00001
!hadoop fs -cat /outputs/part-00002
!hadoop fs -cat /outputs/part-00003
!hadoop fs -cat /outputs/part-00004
!hadoop fs -cat /outputs/part-00005
!hadoop fs -cat /outputs/part-00006

2020-10-29 21:32:34,041 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
cheese	bread	0.2
cheese	coffee	0.2
cheese	eggs	0.2
cheese	milk	0.4
milk	bread	0.14285714285714285
milk	candy	0.14285714285714285
milk	cheese	0.2857142857142857
milk	coffee	0.14285714285714285
milk	coke	0.14285714285714285
milk	eggs	0.14285714285714285
2020-10-29 21:32:39,138 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
coffee	cheese	0.3333333333333333
coffee	eggs	0.3333333333333333
coffee	milk	0.3333333333333333
2020-10-29 21:32:43,200 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
candy	coke	0.5
candy	milk	0.5
2020-10-29 21:32:57,829 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
bread	cheese	0.5
bread	milk	0.5
2020-10-29 21:33:02,175 INFO sasl.S

# Big Data

In [6]:
!hadoop fs -rm -R /outputs
!hadoop jar /opt/hadoop-3.2.1/share/hadoop/tools/lib/hadoop-streaming-3.2.1.jar \
    -D stream.num.map.output.key.fields=2 \
    -D mapreduce.partition.keypartitioner.options=-k1,1 \
    -D mapreduce.job.reduces=25 \
    -file $PWD/mapper.py\
    -file $PWD/reducer.py\
    -mapper mapper.py \
    -reducer reducer.py \
    -input /inputs/data/retail.dat \
    -output /outputs/ \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner

Deleted /outputs
2020-10-29 21:33:38,216 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/training/mapper.py, /training/reducer.py, /tmp/hadoop-unjar7364661987132366336/] [] /tmp/streamjob4101167826217886714.jar tmpDir=null
2020-10-29 21:33:42,376 INFO client.RMProxy: Connecting to ResourceManager at resourcemanager/172.18.0.2:8032
2020-10-29 21:33:45,317 INFO client.AHSProxy: Connecting to Application History server at historyserver/172.18.0.3:10200
2020-10-29 21:33:45,358 INFO client.RMProxy: Connecting to ResourceManager at resourcemanager/172.18.0.2:8032
2020-10-29 21:33:45,359 INFO client.AHSProxy: Connecting to Application History server at historyserver/172.18.0.3:10200
2020-10-29 21:33:46,627 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1604005958251_0002
2020-10-29 21:33:48,142 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localH

In [7]:
!hadoop fs -ls /outputs

Found 26 items
-rw-r--r--   3 root supergroup          0 2020-10-29 21:44 /outputs/_SUCCESS
-rw-r--r--   3 root supergroup    9404342 2020-10-29 21:41 /outputs/part-00000
-rw-r--r--   3 root supergroup    9496011 2020-10-29 21:41 /outputs/part-00001
-rw-r--r--   3 root supergroup    8420049 2020-10-29 21:41 /outputs/part-00002
-rw-r--r--   3 root supergroup    9975371 2020-10-29 21:41 /outputs/part-00003
-rw-r--r--   3 root supergroup    8790278 2020-10-29 21:41 /outputs/part-00004
-rw-r--r--   3 root supergroup    8800016 2020-10-29 21:42 /outputs/part-00005
-rw-r--r--   3 root supergroup    8943834 2020-10-29 21:42 /outputs/part-00006
-rw-r--r--   3 root supergroup    8681714 2020-10-29 21:42 /outputs/part-00007
-rw-r--r--   3 root supergroup    7965952 2020-10-29 21:42 /outputs/part-00008
-rw-r--r--   3 root supergroup    9212994 2020-10-29 21:42 /outputs/part-00009
-rw-r--r--   3 root supergroup    9237805 2020-10-29 21:42 /outputs/part-00010
-rw-r--r--   3 root supergroup    94680

In [8]:
!hadoop fs -cat /outputs/part-00000 | tail
!hadoop fs -cat /outputs/part-00001 | tail
!hadoop fs -cat /outputs/part-00002 | tail
!hadoop fs -cat /outputs/part-00003 | tail
!hadoop fs -cat /outputs/part-00004 | tail
!hadoop fs -cat /outputs/part-00005 | tail
!hadoop fs -cat /outputs/part-00006 | tail
!hadoop fs -cat /outputs/part-00007 | tail
!hadoop fs -cat /outputs/part-00008 | tail
!hadoop fs -cat /outputs/part-00009 | tail


2020-10-29 21:55:34,937 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
9982	38	0.0625
9982	479	0.0625
9982	48	0.0625
9982	4911	0.0625
9982	5782	0.0625
9982	623	0.0625
9982	6340	0.0625
9982	9	0.0625
9982	9223	0.0625
9982	961	0.0625
2020-10-29 21:55:38,220 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
999	991	0.003048780487804878
999	992	0.003048780487804878
999	993	0.018292682926829267
999	994	0.003048780487804878
999	9949	0.003048780487804878
999	995	0.003048780487804878
999	996	0.009146341463414634
999	997	0.003048780487804878
999	998	0.006097560975609756
999	9992	0.003048780487804878
2020-10-29 21:55:42,164 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
9943	987	0.004901960784313725
9984	1026	0.1111111111111111
9984	1543	0.1111111111111111
9984	2563	0.1111111111111111
9984	2870	0.11